# Chat Agent Executor using prebuilt Tool Node


In this example we will build a ReAct Agent that uses tool calling and the prebuilt ToolNode.

## Setup

First we need to install the packages required

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain langchain_openai tavily-python

Next, we need to set API keys for OpenAI (the LLM we will use) and Tavily (the search tool we will use)

In [2]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

Optionally, we can set API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.

In [13]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

## Set up the tools

We will first define the tools we want to use.
For this simple example, we will use a built-in search tool via Tavily.
However, it is really easy to create your own tools - see documentation [here](https://python.langchain.com/v0.2/docs/how_to/custom_tools) on how to do that.

**MODIFICATION**

We don't need a ToolExecutor when using ToolNode.


In [4]:
!pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.9 MB/s eta 0:00:00


In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

## Set up the model

Now we need to load the chat model we want to use.
Importantly, this should satisfy two criteria:

1. It should work with messages. We will represent all agent state in the form of messages, so it needs to be able to work well with them.
2. It should work with tool calling. This means it should be a model that implements `.bind_tools()`.

Note: these model requirements are not requirements for using LangGraph - they are just requirements for this one example.


In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)


After we've done this, we should make sure the model knows that it has these tools available to call.
We can do this by converting the LangChain tools into the format for OpenAI function calling, and then bind them to the model class.


In [7]:
model = model.bind_tools(tools)

## Define the agent state

The main type of graph in `langgraph` is the [StateGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.StateGraph).
This graph is parameterized by a state object that it passes around to each node.
Each node then returns operations to update that state.
These operations can either SET specific attributes on the state (e.g. overwrite the existing values) or ADD to the existing attribute.
Whether to set or add is denoted by annotating the state object you construct the graph with.

For this example, the state we will track will just be a list of messages.
We want each node to just add messages to that list.
Therefore, we will use a `TypedDict` with one key (`messages`) and annotate it so that the `messages` attribute is always added to.


In [8]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

## Define the nodes

We now need to define a few different nodes in our graph.
In `langgraph`, a node can be either a function or a [runnable](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel).
There are two main nodes we need for this:

1. The agent: responsible for deciding what (if any) actions to take.
2. **MODIFICATION** The prebuilt ToolNode, given the list of tools. This will take tool calls from the most recent AIMessage, execute them, and return the result as ToolMessages.

We will also need to define some edges.
Some of these edges may be conditional.
The reason they are conditional is that based on the output of a node, one of several paths may be taken.
The path that is taken is not known until that node is run (the LLM decides).

1. Conditional Edge: after the agent is called, we should either:
   a. If the agent said to take an action, then the function to invoke tools should be called
   b. If the agent said that it was finished, then it should finish
2. Normal Edge: after the tools are invoked, it should always go back to the agent to decide what to do next

Let's define the nodes, as well as a function to decide how what conditional edge to take.


In [9]:
from langgraph.prebuilt import ToolNode


# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If there are no tool calls, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


# Define the function that calls the model
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define the function to execute tools
tool_node = ToolNode(tools)

## Define the graph

We can now put it all together and define the graph!

In [10]:
from langgraph.graph import END, StateGraph, START

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

## Use it!

We can now use it!
This now exposes the [same interface](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel) as all other LangChain runnables.

In [14]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in San Jerónimo de Tunán, Huancayo, Perú")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in San Jerónimo de Tunán, Huancayo, Perú'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Qtzr62qwMXJJy40TCFa5nRBG', 'function': {'arguments': '{"query":"weather in San Jerónimo de Tunán, Huancayo, Perú"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 96, 'total_tokens': 127}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b0abffe81', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6603e0ab-83a9-4425-8ed1-44a76cd6f2d8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Jerónimo de Tunán, Huancayo, Perú'}, 'id': 'call_Qtzr62qwMXJJy40TCFa5nRBG', 'type': 'tool_call'}], usage_metadata={'input_tokens': 96, 'output_tokens': 31, 'total_tokens': 127}),
  ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Hu

This may take a little bit - it's making a few calls behind the scenes.
In order to start seeing some intermediate results as they happen, we can use streaming - see below for more information on that.

## Streaming

LangGraph has support for several different types of streaming.

### Streaming Node Output

One of the benefits of using LangGraph is that it is easy to stream output as it's produced by each node.


In [15]:
inputs = {"messages": [HumanMessage(content="what is the weather in San Jerónimo de Tunán, Huancayo, Perú?")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KB4CvalNwmgFAlyjS8mulPnE', 'function': {'arguments': '{"query":"weather in San Jerónimo de Tunán, Huancayo, Perú"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 97, 'total_tokens': 128}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b0abffe81', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-009c00f5-c991-4816-a384-a6fd04ac7e4a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Jerónimo de Tunán, Huancayo, Perú'}, 'id': 'call_KB4CvalNwmgFAlyjS8mulPnE', 'type': 'tool_call'}], usage_metadata={'input_tokens': 97, 'output_tokens': 31, 'total_tokens': 128})]}

---



Output from node 'action':
---
{'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Huancayo\', \'region\': \'Junin\', \'country\': \'Peru\', \'lat\': -12.07, \'lon\': -75.23, \'tz_id\': \'America/Lima\', \'localtime_epoch\': 1722786599, \'localtime\': \'2024-08-04 10:49\'}, \'current\': {\'last_updated_epoch\': 1722786300, \'last_updated\': \'2024-08-04 10:45\', \'temp_c\': 11.3, \'temp_f\': 52.3, \'is_day\': 1, \'condition\': {\'text\': \'Patchy rain nearby\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/176.png\', \'code\': 1063}, \'wind_mph\': 2.2, \'wind_kph\': 3.6, \'wind_degree\': 250, \'wind_dir\': \'WSW\', \'pressure_mb\': 1019.0, \'pressure_in\': 30.1, \'precip_mm\': 0.01, \'precip_in\': 0.0, \'humidity\': 54, \'cloud\': 85, \'feelslike_c\': 11.3, \'feelslike_f\': 52.3, \'windchill_c\': 11.3, \'windchill_f\': 52.3, \'heatindex_c\': 11.3, \'heatindex_f\': 52.3, \'dewpoint_c\': 2.4, \'dewpoint_f\': 36.3, \'vis_

### Streaming LLM Tokens

You can also access the LLM tokens as they are produced by each node.
In this case only the "agent" node produces LLM tokens.
In order for this to work properly, you must be using an LLM that supports streaming as well as have set it when constructing the LLM (e.g. `ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)`)


In [ ]:
inputs = {"messages": [HumanMessage(content="what is the weather in sf?")]}

async for output in app.astream_log(inputs, include_types=["llm"]):
    # astream_log() yields the requested logs (here LLMs) in JSONPatch format
    for op in output.ops:
        if op["path"] == "/streamed_output/-":
            # this is the output from .stream()
            ...
        elif op["path"].startswith("/logs/") and op["path"].endswith(
            "/streamed_output/-"
        ):
            # because we chose to only include LLMs, these are LLM tokens
            print(op["value"])

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_EdTLEVxQKMLRNv82Yqdcugdy', 'function': {'arguments': '', 'name': 'tavily_search_results_json'}, 'type': 'function'}]} id='run-acf76f4b-c5d0-46a1-a114-75021091719b' invalid_tool_calls=[{'name': 'tavily_search_results_json', 'args': '', 'id': 'call_EdTLEVxQKMLRNv82Yqdcugdy', 'error': 'Malformed args.'}] tool_call_chunks=[{'name': 'tavily_search_results_json', 'args': '', 'id': 'call_EdTLEVxQKMLRNv82Yqdcugdy', 'index': 0}]
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': '{"', 'name': None}, 'type': None}]} id='run-acf76f4b-c5d0-46a1-a114-75021091719b' tool_calls=[{'name': '', 'args': {}, 'id': None}] tool_call_chunks=[{'name': None, 'args': '{"', 'id': None, 'index': 0}]
content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': None, 'function': {'arguments': 'query', 'name': None}, 'type': None}]} id='run-acf76f4b-c5d0-46a1-a114-75021091719b' invalid_tool_calls=[{'n